In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [2]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [3]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [4]:
garbageCollectionDate[14]

'StrathbogieShire-garbage-collection-zones.json'

In [5]:
readInJson(garbageCollectionDate[14]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_20e9ca8f_cad7_4044_a490_c46b86e9ff83.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Monday - Area B', 'rub_day': 'Monday..."
1,Feature,ckan_20e9ca8f_cad7_4044_a490_c46b86e9ff83.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Tuesday - Area B', 'rub_day': 'Tuesd..."


In [10]:
df = readInJson(garbageCollectionDate[14])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [7]:
df.iloc[0,0]

{'name': 'Monday - Area B',
 'rub_day': 'Monday',
 'rub_weeks': '2',
 'rub_start': '2017-05-22',
 'rub_dates': None,
 'rub_name': 'Rubbish - Red Bin',
 'rec_day': 'Monday',
 'rec_weeks': '2',
 'rec_start': '2017-05-15',
 'rec_dates': None,
 'rec_name': 'Recycling - Yellow Bin',
 'grn_day': 'Monday',
 'grn_weeks': '1',
 'grn_start': '2017-05-15',
 'grn_dates': None,
 'grn_name': 'Organics - Green Bin',
 'hw_day': None,
 'hw_weeks': None,
 'hw_start': None,
 'hw_dates': None,
 'info_url': 'https://www.strathbogie.vic.gov.au/images/Waste/Waste_Calendar_Area_B_Monday_2020-2021.pdf',
 'missed_ph': '1800 065 993',
 'mi_style': 'Pen (1,2,0)  Brush (2,16777215,16777215)'}

In [11]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''

    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

df.loc[df.green_colDay == 'Every ', 'green_colDay'] = 'Unavailable'

In [15]:
strathbogie = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

strathbogie.to_csv('../GBD_strathbogiea.csv', index = False)